#### Step 1. function to map ndl tensors to tvm 

In [ ]:
import tvm
from tvm import relax

def convert_tensor_to_tvm(tensor):
    numpy_array = tensor.numpy() 
    return tvm.nd.array(numpy_array)

#### Step 2. Map ndl ops to tvm ops

In [ ]:
from tvm.relax import op
def map_op_to_tvm(op):
    op_mapping = {
        "add": op.add,
        "matmul": op.matmul,
        "relu": op.relu,
        # we'd add any other ops here any ops from ops.mathematic or ops.logarithmic
    }
    if op.__class__.__name__ not in op_mapping:
        raise ValueError(f"Unsupported operation: {op.__class__.__name__}")
    return op_mapping[op.__class__.__name__]

#### Step 3. traverse the ndl graph and generate tvm code, since we have mappings from ndl to tvm for both tensors and ops, we can generate the tvm code from ndl graph

In [ ]:
from tvm.relax import Function, Var, block_builder
from collections import deque

def convert_graph_to_tvm(output_tensor):
    bb = block_builder.BlockBuilder()

    # Map of `Value` nodes to Relax variables
    value_to_var = {}

    # Topological sort of the graph
    def topological_sort(output):
        visited = set()
        topo_order = []

        def dfs(node):
            if node in visited:
                return
            visited.add(node)
            for inp in node.inputs:
                dfs(inp)
            topo_order.append(node)

        dfs(output)
        return reversed(topo_order)  # Reverse for topological order

    # Process the graph in topological order
    topo_order = topological_sort(output_tensor)
    for node in topo_order:
        # Leaf nodes (inputs or constants)
        if node.is_leaf():
            tvm_var = bb.emit_var(relax.Var(node.op.__class__.__name__, shape=node.shape))
            value_to_var[node] = tvm_var
            continue

        # Map the operation to TVM
        tvm_op = map_op_to_tvm(node.op)

        # Get TVM inputs by recursively converting dependencies
        tvm_inputs = [value_to_var[inp] for inp in node.inputs]

        # Emit the Relax operation
        tvm_var = bb.emit(tvm_op(*tvm_inputs))
        value_to_var[node] = tvm_var

    # Create the Relax function
    with bb.function("main"):
        bb.emit_output(value_to_var[output_tensor])
    return bb.get()